# Cleaning the AWS Textract results

In [1]:
from utilities import * 
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [32]:
def duplicate_remover(x1, x2):
    x = [x1, x2]
    x.append(None)
    xx = list(set(x))
    xx.remove(None)
    if len(xx) > 0:
        return xx[0]
    else:
        return None

def get_zip (x):
    if x is None:
        return None, None
    try:
        x = x.replace('\n', ' ')
        zip_ = re.findall(r'\b\d+\b', x)[0]
        state_ = x.replace(zip_, '').strip()
        return state_, zip_
    except: 
        return x.replace('\n', ' '), None
    
def dict_parser (x):
    try:
        x['value']
        return x['value'] , x['confidence']
    except: 
        return None, None

def expand_dict(df, col_list):
    for col_name in col_list:
        df[f'{col_name}_VALUE'], df[f'{col_name}_CONFIDENCE']  = zip(*df[col_name].apply(dict_parser))
    return df

def merge_two_col(x1, x2):
    try:    
        if x1 != False:
            return x1
        if x2 != False:
            return x2
        else:
            print(x1,x2)
            return False
    except:
        pass
    

In [10]:
with open('../dataset/raw/nist/textract_dictionary.pkl', 'rb') as fp:
    result_dict = pickle.load(fp)

In [11]:
df = pd.DataFrame(result_dict)
print(df.shape)
df.head(2)

(2100, 68)


,DATE,CITY,STATE,NAME,ZIP,image_name,STATE ZIP,.DATE,ZAME,in being collected C A.*,...,ML:,CITY-,n,7,-*,THE,00E,hn,t,5
0,"{'value': '08/02/89', 'confidence': 88.9746398...","{'value': 'FLiNT', 'confidence': 86.7397918701...","{'value': 'Mi', 'confidence': 80.10208892822266}","{'value': 'None', 'confidence': nan}","{'value': '48504', 'confidence': 73.8232574462...",f0000_14.png,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'value': '8-7-89', 'confidence': 87.285079956...","{'value': 'Allendale', 'confidence': 89.946899...","{'value': 'MI', 'confidence': 82.12265014648438}","{'value': 'None', 'confidence': nan}","{'value': '49401', 'confidence': 77.6002655029...",f0001_41.png,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
cl_rm = ['n', '7', '-*', 'THE', '00E', 'hn', 't', '5',
':- tentions ...', ',-','1.', 'with', 'S', 'TLI', 'LAME', 'ML:',
 '00', 'mii', 'mL', '--', 'l', '-e', '....', 'for .... in tention',
'- -', 'mLin','sample of handwriting is being collected for use in tection',
'us','m', 'itnl', 'mL:-', 'LU','mLie', '02', 'LO in Loine 1 5. - testime',
'This sample of handwriting is being collected for use in testing computer recognition of hand printed',
'7t receive', 'the', 'NA ME', 'of handwriting is being collected for', 'NAME',
'-', 'mh:-', 'min', 'HANDWRITING SAMPLE FORM', 'iee', '*.', 'NAVE', 'nic',
'inu', '..', 'mL:', '.', 'This', '...', '0', ': A','ZAME',  'in being collected C A.*' ]

df.drop(cl_rm,axis = 1, inplace = True)
print(df.shape)
df.head(2)

(2100, 9)


,DATE,CITY,STATE,ZIP,image_name,STATE ZIP,.DATE,. DATE,CITY-
0,"{'value': '08/02/89', 'confidence': 88.9746398...","{'value': 'FLiNT', 'confidence': 86.7397918701...","{'value': 'Mi', 'confidence': 80.10208892822266}","{'value': '48504', 'confidence': 73.8232574462...",f0000_14.png,NaN,NaN,NaN,NaN
1,"{'value': '8-7-89', 'confidence': 87.285079956...","{'value': 'Allendale', 'confidence': 89.946899...","{'value': 'MI', 'confidence': 82.12265014648438}","{'value': '49401', 'confidence': 77.6002655029...",f0001_41.png,NaN,NaN,NaN,NaN


In [13]:
df.columns

Index(['DATE', 'CITY', 'STATE', 'ZIP', 'image_name', 'STATE ZIP', '.DATE',
       '. DATE', 'CITY-'],
      dtype='object')

In [14]:
df.isnull().sum()

DATE            18
CITY             9
STATE          247
ZIP            392
image_name       0
STATE ZIP     1894
.DATE         2083
. DATE        2099
CITY-         2099
dtype: int64

In [15]:
df.fillna(False, inplace = True)

## Fixing Date:

In [17]:
df["DATE_"] = df.apply(lambda x: merge_two_col(x["DATE"], x[".DATE"]), axis=1)
df["DATE_"] = df.apply(lambda x: merge_two_col(x["DATE_"], x[". DATE"]), axis=1)
print(df[df.DATE_ == False])
df.head(2)

False False
Empty DataFrame
Columns: [DATE, CITY, STATE, ZIP, image_name, STATE ZIP, .DATE, . DATE, CITY-, DATE_]
Index: []


,DATE,CITY,STATE,ZIP,image_name,STATE ZIP,.DATE,. DATE,CITY-,DATE_
0,"{'value': '08/02/89', 'confidence': 88.9746398...","{'value': 'FLiNT', 'confidence': 86.7397918701...","{'value': 'Mi', 'confidence': 80.10208892822266}","{'value': '48504', 'confidence': 73.8232574462...",f0000_14.png,False,False,False,False,"{'value': '08/02/89', 'confidence': 88.9746398..."
1,"{'value': '8-7-89', 'confidence': 87.285079956...","{'value': 'Allendale', 'confidence': 89.946899...","{'value': 'MI', 'confidence': 82.12265014648438}","{'value': '49401', 'confidence': 77.6002655029...",f0001_41.png,False,False,False,False,"{'value': '8-7-89', 'confidence': 87.285079956..."


## Fixing CITY:

In [18]:
df["CITY_"] = df.apply(lambda x: merge_two_col(x["CITY"], x["CITY-"]), axis=1)
print(df.shape)
df.head(2)

False False
False False
False False
False False
False False
False False
False False
False False
(2100, 11)


,DATE,CITY,STATE,ZIP,image_name,STATE ZIP,.DATE,. DATE,CITY-,DATE_,CITY_
0,"{'value': '08/02/89', 'confidence': 88.9746398...","{'value': 'FLiNT', 'confidence': 86.7397918701...","{'value': 'Mi', 'confidence': 80.10208892822266}","{'value': '48504', 'confidence': 73.8232574462...",f0000_14.png,False,False,False,False,"{'value': '08/02/89', 'confidence': 88.9746398...","{'value': 'FLiNT', 'confidence': 86.7397918701..."
1,"{'value': '8-7-89', 'confidence': 87.285079956...","{'value': 'Allendale', 'confidence': 89.946899...","{'value': 'MI', 'confidence': 82.12265014648438}","{'value': '49401', 'confidence': 77.6002655029...",f0001_41.png,False,False,False,False,"{'value': '8-7-89', 'confidence': 87.285079956...","{'value': 'Allendale', 'confidence': 89.946899..."


## parsing dict:

In [20]:
df.drop(['CITY-','. DATE', '.DATE'],axis = 1, inplace = True)
print(df.shape)
df.head(2)

(2100, 8)


,DATE,CITY,STATE,ZIP,image_name,STATE ZIP,DATE_,CITY_
0,"{'value': '08/02/89', 'confidence': 88.9746398...","{'value': 'FLiNT', 'confidence': 86.7397918701...","{'value': 'Mi', 'confidence': 80.10208892822266}","{'value': '48504', 'confidence': 73.8232574462...",f0000_14.png,False,"{'value': '08/02/89', 'confidence': 88.9746398...","{'value': 'FLiNT', 'confidence': 86.7397918701..."
1,"{'value': '8-7-89', 'confidence': 87.285079956...","{'value': 'Allendale', 'confidence': 89.946899...","{'value': 'MI', 'confidence': 82.12265014648438}","{'value': '49401', 'confidence': 77.6002655029...",f0001_41.png,False,"{'value': '8-7-89', 'confidence': 87.285079956...","{'value': 'Allendale', 'confidence': 89.946899..."


In [22]:
df = expand_dict(df, ['DATE_', 'CITY_', 'STATE','ZIP'])
print(df.shape)
df.head(2)

(2100, 16)


,DATE,CITY,STATE,ZIP,image_name,STATE ZIP,DATE_,CITY_,DATE__VALUE,DATE__CONFIDENCE,CITY__VALUE,CITY__CONFIDENCE,STATE_VALUE,STATE_CONFIDENCE,ZIP_VALUE,ZIP_CONFIDENCE
0,"{'value': '08/02/89', 'confidence': 88.9746398...","{'value': 'FLiNT', 'confidence': 86.7397918701...","{'value': 'Mi', 'confidence': 80.10208892822266}","{'value': '48504', 'confidence': 73.8232574462...",f0000_14.png,False,"{'value': '08/02/89', 'confidence': 88.9746398...","{'value': 'FLiNT', 'confidence': 86.7397918701...",08/02/89,88.97464,FLiNT,86.739792,Mi,80.102089,48504,73.823257
1,"{'value': '8-7-89', 'confidence': 87.285079956...","{'value': 'Allendale', 'confidence': 89.946899...","{'value': 'MI', 'confidence': 82.12265014648438}","{'value': '49401', 'confidence': 77.6002655029...",f0001_41.png,False,"{'value': '8-7-89', 'confidence': 87.285079956...","{'value': 'Allendale', 'confidence': 89.946899...",8-7-89,87.28508,Allendale,89.946899,MI,82.122650,49401,77.600266


In [23]:
df.columns

Index(['DATE', 'CITY', 'STATE', 'ZIP', 'image_name', 'STATE ZIP', 'DATE_',
       'CITY_', 'DATE__VALUE', 'DATE__CONFIDENCE', 'CITY__VALUE',
       'CITY__CONFIDENCE', 'STATE_VALUE', 'STATE_CONFIDENCE', 'ZIP_VALUE',
       'ZIP_CONFIDENCE'],
      dtype='object')

In [24]:
df_out = df[['image_name', 
    'DATE__VALUE', 'DATE__CONFIDENCE',
    'CITY__VALUE', 'CITY__CONFIDENCE', 
    'STATE_VALUE', 'STATE_CONFIDENCE', 
    'ZIP_VALUE', 'ZIP_CONFIDENCE']]
print(df_out.shape)
print(df_out.isnull().sum())

(2100, 9)
image_name            0
DATE__VALUE           0
DATE__CONFIDENCE      7
CITY__VALUE           8
CITY__CONFIDENCE     17
STATE_VALUE         247
STATE_CONFIDENCE    282
ZIP_VALUE           392
ZIP_CONFIDENCE      431
dtype: int64


In [25]:
df_out.replace(to_replace = False, value = None, inplace = True)

image_name          None
DATE__VALUE         None
DATE__CONFIDENCE    None
CITY__VALUE         None
CITY__CONFIDENCE    None
STATE_VALUE         None
STATE_CONFIDENCE    None
ZIP_VALUE           None
ZIP_CONFIDENCE      None
dtype: object

In [26]:
df_out.isnull().sum()

image_name            0
DATE__VALUE           0
DATE__CONFIDENCE      7
CITY__VALUE           8
CITY__CONFIDENCE     17
STATE_VALUE         247
STATE_CONFIDENCE    282
ZIP_VALUE           392
ZIP_CONFIDENCE      431
dtype: int64

In [27]:
df_out

,image_name,DATE__VALUE,DATE__CONFIDENCE,CITY__VALUE,CITY__CONFIDENCE,STATE_VALUE,STATE_CONFIDENCE,ZIP_VALUE,ZIP_CONFIDENCE
0,f0000_14.png,08/02/89,88.974640,FLiNT,86.739792,Mi,80.102089,48504,73.823257
1,f0001_41.png,8-7-89,87.285080,Allendale,89.946899,MI,82.122650,49401,77.600266
2,f0002_01.png,8-3-89,81.038841,MINDEN CITY,80.873611,mi,35.721474,48456,37.457157
3,f0003_42.png,08-03-89,87.377403,Charlotte,80.985222,MI,77.612007,48813,71.598038
4,f0004_09.png,08-03-89,89.594048,Holland,80.345238,Mi,62.762756,49424,58.702671
...,...,...,...,...,...,...,...,...,...
2095,f2095_37.png,10/8/89,84.937805,LITTLETON,81.430542,Co,71.639107,80122,63.081978
2096,f2096_00.png,9-29-89,84.306885,FARGO,72.930412,N.D.,69.265854,58102,63.405502
2097,f2097_31.png,9.2589,85.472717,Belarade The,72.343956,Mt,59.005791,59714,61.639637
2098,f2098_02.png,10/04/89,88.015785,"Burbank,",87.720345,S.D.,85.049950,57010,74.364502


In [33]:
df['CLEAN_STATE_VALUE'], df['CLEAN_ZIP_VALUE'] = zip(*(df.STATE_VALUE.apply(get_zip)))
print(df.shape)
df.head(2)

(2100, 18)


,DATE,CITY,STATE,ZIP,image_name,STATE ZIP,DATE_,CITY_,DATE__VALUE,DATE__CONFIDENCE,CITY__VALUE,CITY__CONFIDENCE,STATE_VALUE,STATE_CONFIDENCE,ZIP_VALUE,ZIP_CONFIDENCE,CLEAN_STATE_VALUE,CLEAN_ZIP_VALUE
0,"{'value': '08/02/89', 'confidence': 88.9746398...","{'value': 'FLiNT', 'confidence': 86.7397918701...","{'value': 'Mi', 'confidence': 80.10208892822266}","{'value': '48504', 'confidence': 73.8232574462...",f0000_14.png,False,"{'value': '08/02/89', 'confidence': 88.9746398...","{'value': 'FLiNT', 'confidence': 86.7397918701...",08/02/89,88.97464,FLiNT,86.739792,Mi,80.102089,48504,73.823257,Mi,None
1,"{'value': '8-7-89', 'confidence': 87.285079956...","{'value': 'Allendale', 'confidence': 89.946899...","{'value': 'MI', 'confidence': 82.12265014648438}","{'value': '49401', 'confidence': 77.6002655029...",f0001_41.png,False,"{'value': '8-7-89', 'confidence': 87.285079956...","{'value': 'Allendale', 'confidence': 89.946899...",8-7-89,87.28508,Allendale,89.946899,MI,82.122650,49401,77.600266,MI,None


In [34]:
df["ZIP_VALUE_FINAL"] = df.apply(lambda x: duplicate_remover(x["ZIP_VALUE"], x["CLEAN_ZIP_VALUE"]), axis=1)
df["STATE_VALUE_FINAL"] = df.apply(lambda x: duplicate_remover(x["STATE_VALUE"], x["CLEAN_STATE_VALUE"]), axis=1)

In [35]:
df_out = df[['image_name', 
    'DATE__VALUE', 'DATE__CONFIDENCE',
    'CITY__VALUE', 'CITY__CONFIDENCE', 
    'CLEAN_STATE_VALUE', 'STATE_CONFIDENCE', 
    'ZIP_VALUE_FINAL', 'ZIP_CONFIDENCE']].copy()
print(df_out.shape)
# df_out = df_out[df_out['ZIP_VALUE_FINAL'].notna()]
print(df_out.shape)
print(df_out.isnull().sum())
df_out.head(2)

(2100, 9)
(2100, 9)
image_name             0
DATE__VALUE            0
DATE__CONFIDENCE       7
CITY__VALUE            8
CITY__CONFIDENCE      17
CLEAN_STATE_VALUE    247
STATE_CONFIDENCE     282
ZIP_VALUE_FINAL      215
ZIP_CONFIDENCE       431
dtype: int64


,image_name,DATE__VALUE,DATE__CONFIDENCE,CITY__VALUE,CITY__CONFIDENCE,CLEAN_STATE_VALUE,STATE_CONFIDENCE,ZIP_VALUE_FINAL,ZIP_CONFIDENCE
0,f0000_14.png,08/02/89,88.97464,FLiNT,86.739792,Mi,80.102089,48504,73.823257
1,f0001_41.png,8-7-89,87.28508,Allendale,89.946899,MI,82.122650,49401,77.600266


In [36]:
df_out.rename(columns={"DATE__VALUE": "DATE", 
                       "CITY__VALUE": "CITY", 
                       "CLEAN_STATE_VALUE": "STATE",
                      "ZIP_VALUE_FINAL":"ZIP"}, 
              errors="raise", 
              inplace = True)
print(df_out.shape)
df_out.head(2)

(2100, 9)


,image_name,DATE,DATE__CONFIDENCE,CITY,CITY__CONFIDENCE,STATE,STATE_CONFIDENCE,ZIP,ZIP_CONFIDENCE
0,f0000_14.png,08/02/89,88.97464,FLiNT,86.739792,Mi,80.102089,48504,73.823257
1,f0001_41.png,8-7-89,87.28508,Allendale,89.946899,MI,82.122650,49401,77.600266


In [37]:
df_out.isnull().sum()

image_name            0
DATE                  0
DATE__CONFIDENCE      7
CITY                  8
CITY__CONFIDENCE     17
STATE               247
STATE_CONFIDENCE    282
ZIP                 215
ZIP_CONFIDENCE      431
dtype: int64

In [38]:
df_out.to_csv('../dataset/raw/nist/aws_cleaned.csv', index = False)